In [ ]:
'''
Created by Karuna Puri on 14/05/2020
'''

import requests
import csv
from bs4 import BeautifulSoup

import urllib.parse
url = "https://materials.springer.com/search?searchTerm=BENZENE&propertyFacet="
parsed = urllib.parse.urlparse(url)
path = parsed.path
page_views = 1
unique_views = 1


if path=='/textsearch':
    search_type = 'Marklogic'
    page = requests.get(url)
    
elif path =='/search':
    search_type = 'Graph'
    page = requests.get(url)
    

else:
    page = None
    search_type = '-'
    user_search_term = '-'
    search_term_contextual = '-'
    search_term_non_contextual = '-'
    total_graph_count = 0
    total_ml_count = 0
    search_category = '-'
    fields=['Search Term', 'Results Served From','Context','Non-contextual','Graph Results', 'Marklogic Results', 'Total Views', 'Unique Views', 'Search Category', 'URL']
    with open(r'Analytics.csv', 'a', newline="\n") as f:
        writer = csv.writer(f)
        writer.writerow([user_search_term, search_type, search_term_contextual, search_term_non_contextual, total_graph_count, total_ml_count, page_views, unique_views, search_category, url])
    
    

if page is not None:
    soup = BeautifulSoup(page.text, 'html.parser')
else:
    soup = BeautifulSoup(features='xml')
    
interactive_tooltip = soup.find("div", { "class" :'smi-tooltip'})
interactive_label = soup.find("span", { "class" :'smi-label'})

if soup.find(class_='facet-count') is not None: 

    result_count_list = soup.find(class_='facet-count')
    ml_result_list = soup.find("div", { "id" : "alternate_search_top" })

    if ml_result_list is not None and "If you didn't find what you were looking for," in ml_result_list.text:
        result_count_list_items = result_count_list.find_all('strong')
    elif ml_result_list is None:
        result_count_list_items = result_count_list.find_all('strong')
    else:
        result_count_list_items = ml_result_list.find_all('a')
    

else:
    result_count_list = soup.find('')
    result_count_list_items = []
    ml_result_list=soup.find('')
 
    
if ml_result_list is None:
    ml_result_list_items = []
elif ml_result_list is not None and "Too many results?" in ml_result_list.text:
    ml_result_list_items = result_count_list.find_all('strong')

else:
    ml_result_list_items = ml_result_list.find_all('a')
    
for result_count in result_count_list_items:
    total_result_count = result_count.contents[0].strip()

for ml_result_count in ml_result_list_items:
    ml_total_result_count = ml_result_count.contents[0].strip()
    

    
import urllib.parse
    
parsed = urllib.parse.urlparse(url)
params = urllib.parse.parse_qsl(parsed.query)

for x,y in params:
    if 'searchTerm' in x:
        user_search_term = y
    else:
        user_search_term = "-"
        
        
contextual_search_term = soup.find_all(class_='context')
selected_facet_search_term = soup.find_all(class_='selected-facets')
non_contextual_term = soup.find_all(class_='non-contextual')


if contextual_search_term is None and selected_facet_search_term is not None:
    search_term = ([search_term.text for search_term in selected_facet_search_term])
    search_term_contextual = '-'
    search_term_non_contextual = '-'
    
elif contextual_search_term is not None:

    if soup.find(class_='context'):
        search_term_contextual = ([search_term_contextual.text for search_term_contextual in contextual_search_term])
    else:
        search_term_contextual = '-'
    if soup.find(class_='non-contextual'):
        search_term_non_contextual = ([search_term_non_contextual.text for search_term_non_contextual in non_contextual_term])
    else:
        search_term_non_contextual = '-'

else:
    search_term_contextual = '-'
    search_term_non_contextual = '-'
    user_search_term = '-'
    
for a_tag in soup.find_all('a', href=True):
    href_tag = (a_tag['href'])
    if 'Back to Focused Search.' in a_tag.text:
        graph_url = a_tag['href']
    elif 'see more results.' in a_tag.text:
        ml_url = a_tag['href']
        
fields=['Search Term', 'Results Served From','Context','Non-contextual','Graph Results', 'Marklogic Results', 'Total Views', 'Unique Views', 'Search Category', 'URL']
with open(r'Analytics.csv', 'a', newline="\n") as f:
    writer = csv.writer(f)

    
    
    if result_count_list is None and ml_result_list is None :
        marklogic_results = 0
        total_result_count = 0
        search_category = '-'
        writer.writerow(["\n"])
        writer.writerow([user_search_term, search_type, search_term_contextual, search_term_non_contextual, total_result_count, marklogic_results, page_views, unique_views, search_category, url]) 
    
    elif ml_result_list is None and result_count_list is not None:
        marklogic_results = 0
        
        if interactive_tooltip is not None and interactive_label is not None:
            search_category = 'Interactive and Static'
        else:
            search_category = 'Static'
        writer.writerow(["\n"])   
        writer.writerow([user_search_term,search_type, search_term_contextual, search_term_non_contextual, total_result_count, marklogic_results, page_views, unique_views, search_category, url]) 
        
    
    else:
        for ml_result in ml_result_list_items:
            marklogic_results = ml_result.contents[0].strip()
            search_category = 'Interactive and Static'
            
            if 'Back to Focused Search.' in total_result_count: 
                
                graph_page = requests.get(graph_url)
                soup = BeautifulSoup(graph_page.text, 'html.parser')
                if soup.find(class_='facet-count') is not None:
                    graph_result_count = soup.find(class_='facet-count')
                    graph_result=graph_result_count.find_all('strong')
                    for graph_count in graph_result:
                        total_graph_count = graph_count.contents[0].strip()
                if interactive_tooltip is not None and interactive_label is not None:
                    search_category = 'Interactive and Static'
                else:
                    search_category = 'Static'
                        
            else:
                total_graph_count=total_result_count 
                
            if 'see more results.' in marklogic_results: 
                ml_page = requests.get(ml_url)
                soup = BeautifulSoup(ml_page.text, 'html.parser')
                if soup.find(class_='facet-count') is not None:
                    ml_result_count = soup.find(class_='facet-count')
                    ml_result=ml_result_count.find_all('strong')
                    for ml_count in ml_result:
                        total_ml_count = ml_count.contents[0].strip()

            else:
                total_ml_count=marklogic_results
                
            
            writer.writerow(["\n"])
            writer.writerow([user_search_term, search_type, search_term_contextual, search_term_non_contextual, total_graph_count, total_ml_count, page_views, unique_views, search_category, url])


f.close()